## XGBoost Tuning from Feature Pipeline 

This notebook starts from `data/samples/train.parquet` and `data/samples/val.parquet` to reduce upload time.
It performs:
1. Feature engineering via project `FeatureBuilder`
2. `RandomizedSearchCV` tuning for XGBoost
3. Validation ROC-AUC reporting

In [4]:
! git clone https://github.com/AnujKumar-1628/lending-club-credit-risk


Cloning into 'lending-club-credit-risk'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 321 (delta 5), reused 12 (delta 2), pack-reused 300 (from 1)
Receiving objects: 100% (321/321), 948.78 KiB | 829.00 KiB/s, done.
Resolving deltas: 100% (103/103), done.


In [1]:
%cd lending-club-credit-risk

/content/lending-club-credit-risk


In [2]:
!pwd
!ls


/content/lending-club-credit-risk
api		       LICENSE	  requirements-dev.txt	src
Dockerfile	       models	  requirements.txt	template.py
docs		       notebooks  scripts		tests
LCDataDictionary.xlsx  README.md  setup.py


In [4]:
!python -m pip install -q -e .

  Preparing metadata (setup.py) ... done


In [3]:
!python -m pip install -q -r requirements.txt

In [5]:
import credit_risk
print("OK:", credit_risk.__file__)


OK: /content/lending-club-credit-risk/src/credit_risk/__init__.py


In [6]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import json
import joblib
import pandas as pd
import xgboost as xgb
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

from credit_risk.features.build_features import FeatureBuilder


In [7]:
!mkdir -p data/samples

In [8]:
train_path = Path('data/samples/train.parquet')
val_path = Path('data/samples/val.parquet')

if not train_path.exists() or not val_path.exists():
    raise FileNotFoundError(
        'Missing sample files. Expected: data/samples/train.parquet and data/samples/val.parquet'
    )

train_df = pd.read_parquet(train_path)
val_df = pd.read_parquet(val_path)

print('Train shape:', train_df.shape)
print('Validation shape:', val_df.shape)

Train shape: (941716, 30)
Validation shape: (201796, 30)


In [9]:

feature_builder = FeatureBuilder()
X_train, y_train = feature_builder.build_features(train_df, fit=True)
X_val, y_val = feature_builder.build_features(val_df, fit=False)

print('X_train:', X_train.shape)
print('X_val:', X_val.shape)

2026-02-18 11:13:05 | INFO | credit_risk.features.build_features | Building features


INFO:credit_risk.features.build_features:Building features


2026-02-18 11:13:15 | INFO | credit_risk.features.build_features | Building features


INFO:credit_risk.features.build_features:Building features


X_train: (941716, 135)
X_val: (201796, 135)


In [13]:
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    tree_method='hist',
    device='cuda',
    random_state=42,
    n_jobs=-1,
)

param_distributions = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(3, 9),
    'learning_rate': loguniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 12),
    'gamma': uniform(0.0, 5.0),
    'reg_alpha': loguniform(1e-8, 1e-1),
    'reg_lambda': loguniform(1e-3, 1e2),
}

cv = TimeSeriesSplit(n_splits=3)

search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_distributions,
    n_iter=30,
    scoring='roc_auc',
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=1,
    return_train_score=True,
)

search

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBClassifier...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'colsample_bytree': <scipy.stats....x7e31076b6d20>, 'gamma': <scipy.stats....x7e31076b7c20>, 'learning_rate': <scipy.stats....x7e31076b5d60>, 'max_depth': <scipy.stats....x7e31076b5820>, ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",30
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for

In [14]:
search.fit(X_train, y_train)

print('Best CV ROC-AUC:', round(search.best_score_, 6))
print('Best Params:', search.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END colsample_bytree=0.749816047538945, gamma=4.75357153204958, learning_rate=0.08960785365368121, max_depth=7, min_child_weight=5, n_estimators=814, reg_alpha=1.3207763079904522e-05, reg_lambda=0.0031613645505296157, subsample=0.7836995567863468; total time=   5.2s
[CV] END colsample_bytree=0.749816047538945, gamma=4.75357153204958, learning_rate=0.08960785365368121, max_depth=7, min_child_weight=5, n_estimators=814, reg_alpha=1.3207763079904522e-05, reg_lambda=0.0031613645505296157, subsample=0.7836995567863468; total time=   8.8s
[CV] END colsample_bytree=0.749816047538945, gamma=4.75357153204958, learning_rate=0.08960785365368121, max_depth=7, min_child_weight=5, n_estimators=814, reg_alpha=1.3207763079904522e-05, reg_lambda=0.0031613645505296157, subsample=0.7836995567863468; total time=  13.9s
[CV] END colsample_bytree=0.7334834444556088, gamma=0.7143340896097039, learning_rate=0.07027897634919636, max_depth=7, min

In [18]:
best_model = search.best_estimator_
val_prob = best_model.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_prob)
print('Validation ROC-AUC (holdout):', round(val_auc, 6))

Validation ROC-AUC (holdout): 0.705311
